In [ ]:
#Main Part
if __name__ == '__main__':

    import multiprocessing      #for multiprocessing
    from functions import *
    import matplotlib.pyplot as plt
    import pandas as pd
    import copy

    p_conn, c_conn =  multiprocessing.Pipe()
    procs = []

    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(1, 1, 1)
    
    solar_data = pd.DataFrame(pd.read_csv('C:/Users/mhkim/OneDrive/문서/GitHub/Solar_Nbody_system/solarsystem.csv', sep=','))
    solar_data.set_index('name',inplace=True,drop=True)
    
    solsy = Solarsystem(solar_data)      #call solarsystem

    t_end =  31600000                   #initial time setting
    dt = 1.3e6                          #step size
    end = int(t_end/dt)+1
    
    for i in range(end-1):
        dummy = copy.deepcopy(solsy)
        for k in range(1,len(solsy)):
            #plt.scatter(solsy[k].position[0],solsy[k].position[1])
            p_conn.send(solsy)
            p = multiprocessing.Process(target=update, args=(c_conn,dummy,solsy[k],dt,k))    
            p.start()
            print(solsy.position())
            procs.append(p)

        for p in procs:
            p.join()
            
   
    plt.show()


    